## import library 

In [395]:
import pandas as pd
import numpy as np
import math

ModuleNotFoundError: No module named 'xgboost'

## 讀資料

In [426]:
class LogisticRegression(object):
    def __init__(self):
        self.w = None
        self.X = None
        self.y = None
        
        
    def train(self,X,y,batch_size=None,epoch=1000,l_rate=0.3):
        self.X = np.hstack((np.ones((X.shape[0],1)),X))
        self.y = y
        self.w = np.zeros( (self.X.shape[1],1) )
        s_gra = np.zeros( (self.X.shape[1],1) )
        
        num_data = X.shape[0]
        
        if not batch_size:
            batch_size = self.X.shape[0]
        
        num_batch = math.ceil(num_data/batch_size)
        X = self.X
        y = self.y
        
        for i in range(epoch):
            # w,y_plum,s_gra = self.__update(l_rate,s_gra)
            
            for i in range(num_batch):
                
                X_batch = X[i*batch_size:min((i+1)*batch_size,num_data)]
                y_batch = y[i*batch_size:min((i+1)*batch_size,num_data)]
                hypo = np.dot(X_batch,self.w)                  
                y_predict = self.sigmoid(hypo)
                
                
                error = y_batch - y_predict

                gra= -1*np.dot(X_batch.T,error)

                ###### 內積有問題
#                 print(X_batch[:3].T)
#                 print(error[:3])
#                 print(X_batch[:3].T.dot(error[:3]))
                s_gra = s_gra + gra**2  
#                 print("s_gra:{}".format(s_gra))
                self.w = self.w - l_rate/np.sqrt(s_gra)*gra
                

        
    def softmax(self,z):
        z = np.exp(z)
        total = np.sum(z)
        return z/total
    
    def cost_function(self,y_hat,y):
        cost_sum=0
        for i,val in enumerate(y_hat):
            if val[0]==1:
                cost_sum+=y[i][0]
            elif val[0]==0:
                cost_sum+=(1-y[i][0])
                
        return -1*cost_sum
                
        
    def cost_function_derivative(self,X,y_hat,y_sig):
        diff = y_hat - y_sig
        grad = -1*np.dot(X.T,diff)
        return grad
        
    
        
    def predict(self,test_X):
        test_X = np.hstack((np.ones((test_X.shape[0],1)),test_X))
        y_plum = np.dot(test_X,self.w)
        #temp_y = self.sigmoid(y_plum)
        res = [1 if i>=0 else 0 for i in y_plum]
        #res = [1 if i>=0.5 else 0 for i in temp_y]

        return res

    def sigmoid(self,z):
        res = 1/(1.0+np.exp(-z))
        return np.clip(res,0.00000000000001,0.99999999999999)

    
    @staticmethod
    def standarize(df):
        res = df
        cols=res.columns
        for col in cols:
            max_num = res[col].max()
            min_num = res[col].min()
            diff = max_num-min_num
            res[col] = (res[col]-min_num)/diff
        return res
    
    @staticmethod
    def standarize_std(df):
        res=df
        cols = res.columns
        for col in cols:
            std = np.std(res[col])
            mean = np.mean(res[col])
            res[col]=(res[col]-mean)/std
        return res
        

In [369]:
a= np.array([[1,2,3],[3,2,7],[1,2,1],[4,2,5]])
b = np.array([[1],[0],[3],[1]])

#np.sum(a,axis=1)
# lr = LogisticRegression()
# lr.train(a,b)

# lr.predict(a)
#1e-7+1
# b.reshape((-1,1))
# for i,val in enumerate(b):
#     print(val[0])
#np.log(0.01)

c = np.array([1,2,3])
d = np.array([2,3,3])

e = np.zeros((4,1))
#np.sum(a)
(e+b)

/Users/phx/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:20: RuntimeWarning: divide by zero encountered in true_divide


array([[ 3.],
       [inf],
       [ 1.],
       [ 3.]])

In [202]:
income = pd.read_csv("./train.csv")
test = pd.read_csv("./test.csv")



In [203]:
income.head()


,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [204]:
print(income.shape)
print(test.shape)

(32561, 15)
(16281, 14)


In [205]:
y_train = income["income"]==' <=50K'
y_train = np.array([0 if i else 1 for i in y_train]).reshape((-1,1))

X_train = income.drop("income",axis=1)
X_train.shape


(32561, 14)

In [206]:
test.shape

(16281, 14)

In [207]:
#X_train.iloc[32560]
all_data = pd.concat([X_train,test],axis=0)

## 清理資料  
清理後train有106 columns

In [208]:
income.columns



Index(['age', 'workclass', 'fnlwgt', 'education', 'education_num',
       'marital_status', 'occupation', 'relationship', 'race', 'sex',
       'capital_gain', 'capital_loss', 'hours_per_week', 'native_country',
       'income'],
      dtype='object')

In [209]:
discrete_col = ['workclass','education','marital_status','occupation', 'relationship', 'race','native_country']
continuous_col = ['age','fnlwgt','sex','capital_gain', 'capital_loss','hours_per_week']

In [210]:
discrete = pd.get_dummies(all_data[discrete_col])
continuous = all_data[continuous_col]
data = pd.concat([continuous,discrete],axis=1)



In [211]:
data.head()

,age,fnlwgt,sex,capital_gain,capital_loss,hours_per_week,workclass_ ?,workclass_ Federal-gov,workclass_ Local-gov,workclass_ Never-worked,...,native_country_ Portugal,native_country_ Puerto-Rico,native_country_ Scotland,native_country_ South,native_country_ Taiwan,native_country_ Thailand,native_country_ Trinadad&Tobago,native_country_ United-States,native_country_ Vietnam,native_country_ Yugoslavia
0,39,77516,Male,2174,0,40,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,50,83311,Male,0,0,13,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,38,215646,Male,0,0,40,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,53,234721,Male,0,0,40,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,28,338409,Female,0,0,40,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [212]:
sex_mapping={
    ' Male':0,
    ' Female':1
}

data["sex"]=data["sex"].map(sex_mapping)



In [425]:
data.head()


,age,fnlwgt,sex,capital_gain,capital_loss,hours_per_week,workclass_ ?,workclass_ Federal-gov,workclass_ Local-gov,workclass_ Never-worked,...,native_country_ Portugal,native_country_ Puerto-Rico,native_country_ Scotland,native_country_ South,native_country_ Taiwan,native_country_ Thailand,native_country_ Trinadad&Tobago,native_country_ United-States,native_country_ Vietnam,native_country_ Yugoslavia
0,0.301370,0.044131,0.0,0.02174,0.0,0.397959,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,0.452055,0.048052,0.0,0.00000,0.0,0.122449,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,0.287671,0.137581,0.0,0.00000,0.0,0.397959,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,0.493151,0.150486,0.0,0.00000,0.0,0.397959,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,0.150685,0.220635,1.0,0.00000,0.0,0.397959,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## 標準化standarize

In [421]:
lr = LogisticRegression()
data=lr.standarize(data)


In [422]:
X_train = data.iloc[:income.shape[0],:].values
X_test = data.iloc[income.shape[0]:,:].values


In [423]:

lr.train(X_train,y_train,batch_size=100,epoch=1)
predictions=lr.predict(X_test)

/Users/phx/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:42: RuntimeWarning: divide by zero encountered in true_divide
/Users/phx/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:42: RuntimeWarning: invalid value encountered in multiply


In [419]:
ans=pd.read_csv("correct_answer.csv")
ans.head()

,id,label
0,1,0
1,2,0
2,3,1
3,4,1
4,5,0


In [420]:
ans_list=list(ans["label"])
count=0
for i,val in enumerate(ans_list):
    if val == predictions[i]:
         count+=1
rate = count/len(ans_list)
print(rate)

0.7637737239727289


In [106]:
idx = [ i+1 for i,val in enumerate(predictions)]

In [81]:
output_df={
    "id":idx,
    "label":predictions
}

submission = pd.DataFrame(output_df)
submission.to_csv("predict.csv",index=False)

In [47]:
predictions.index

TypeError: index() takes at least 1 argument (0 given)

## 用高斯分佈去預測 

In [ ]:
class Guassian_distribution(object):
    self.__init__(self):
        
    

## 用keras跑

In [18]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.utils import np_utils

/Users/phx/anaconda/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [19]:
model = Sequential()

model.add(Dense(units=100, input_dim=106, kernel_initializer='normal', activation='relu')) 

model.add(Dense(units=2, kernel_initializer='normal', activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy']) 

In [20]:
temp = np_utils.to_categorical(y_train,num_classes=2)

In [21]:
model.fit(X_train,temp,batch_size=100,epochs=20)


Epoch 1/20
32561/32561 [==============================] - 1s 25us/step - loss: 0.3868 - acc: 0.8176
Epoch 2/20
32561/32561 [==============================] - 1s 19us/step - loss: 0.3368 - acc: 0.8433
Epoch 3/20
32561/32561 [==============================] - 1s 20us/step - loss: 0.3277 - acc: 0.8466
Epoch 4/20
32561/32561 [==============================] - 1s 20us/step - loss: 0.3217 - acc: 0.8487
Epoch 5/20
32561/32561 [==============================] - 1s 20us/step - loss: 0.3163 - acc: 0.8530
Epoch 6/20
32561/32561 [==============================] - 1s 19us/step - loss: 0.3128 - acc: 0.8534
Epoch 7/20
32561/32561 [==============================] - 1s 20us/step - loss: 0.3101 - acc: 0.8547
Epoch 8/20
32561/32561 [==============================] - 1s 20us/step - loss: 0.3078 - acc: 0.8567
Epoch 9/20
32561/32561 [==============================] - 1s 20us/step - loss: 0.3062 - acc: 0.8580
Epoch 10/20
32561/32561 [==============================] - 1s 20us/step - loss: 0.3041 - acc: 0.8586

In [46]:
model.evaluate(X_train,temp)

32561/32561 [==============================] - 1s 21us/step


[0.28513212386998205, 0.8669266914461722]

In [47]:
ans_list=list(ans["label"])

In [49]:
ans_temp = np_utils.to_categorical(ans_list,num_classes=2)

In [23]:
res = model.predict(X_test)

In [24]:
res

array([[9.99562919e-01, 4.37084760e-04],
       [8.85397553e-01, 1.14602394e-01],
       [6.47005975e-01, 3.52993995e-01],
       ...,
       [2.58418739e-01, 7.41581202e-01],
       [5.95390916e-01, 4.04609114e-01],
       [1.98494896e-01, 8.01505029e-01]], dtype=float32)

In [61]:
def predict(test_X):
    test_X = np.hstack((np.ones((test_X.shape[0],1)),test_X))
    y_plum = np.dot(test_X,self.__w)
    temp_y = self.sigmoid(y_plum)
    res = [1 if i>=0.5 else 0 for i in temp_y]


    return res


(32561, 2)

In [60]:
X_test.shape

(16281, 106)

In [59]:
X_train

array([[0.30136986, 0.04413121, 0.        , ..., 1.        , 0.        ,
        0.        ],
       [0.45205479, 0.04805174, 0.        , ..., 1.        , 0.        ,
        0.        ],
       [0.28767123, 0.13758131, 0.        , ..., 1.        , 0.        ,
        0.        ],
       ...,
       [0.56164384, 0.09446153, 1.        , ..., 1.        , 0.        ,
        0.        ],
       [0.06849315, 0.12800425, 0.        , ..., 1.        , 0.        ,
        0.        ],
       [0.47945205, 0.18648211, 1.        , ..., 1.        , 0.        ,
        0.        ]])